# 2. Simple LIF neural network in basic clock-driven algorithm
No delays

No refractory periods

No inhibitory neurons

In [1]:
using Plots, StructArrays, LinearAlgebra

In [2]:
const total_time = 10  # seconds
const τ = 50.  # decay rate
const θ = 1.  # threshold potential
const ρ = 0.  # baseline potential
const dt = 0.1  # timestep
const alpha = exp(-dt/τ)
const total_step = Int(total_time / dt)

100

In [3]:
mutable struct Neuron
    V  # Neuron somatic membrane potential
    spike_out::Array{Float32}
end
Neuron() = Neuron(0, Array{Float32, 1}(undef, total_step))

Neuron

In [4]:
l1_n = 5
l2_n = 3

l1 = Vector{Neuron}()
l2 = Vector{Neuron}()

for i in 1:l1_n
    push!(l1, Neuron())
end
for i in 1:l2_n
    push!(l2, Neuron())
end
l1

5-element Vector{Neuron}:
 Neuron(0, Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 Neuron(0, Float32[-9.109722f14, 4.583f-41, -9.109765f14, 4.583f-41, 1.4f-44, 0.0, 1.5f-44, 0.0, -9.109808f14, 4.583f-41  …  -9.11071f14, 4.583f-41, 4.8f-44, 0.0, 5.2f-44, 0.0, -9.110753f14, 4.583f-41, -9.110796f14, 4.583f-41])
 Neuron(0, Float32[1.3f-44, 0.0, 1.3f-44, 0.0, -9.165342f14, 4.583f-41, -9.165385f14, 4.583f-41, 1.4f-44, 0.0  …  4.6f-44, 0.0, -9.166287f14, 4.583f-41, -9.16633f14, 4.583f-41, 4.8f-44, 0.0, 5.2f-44, 0.0])
 Neuron(0, Float32[1.0f-45, 0.0, 3.0f-45, 0.0, 1.0f-45, 0.0, 1.0f-45, 0.0, 1.0f-45, 0.0  …  1.0f-45, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 Neuron(0, Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [21]:
synapse = randn(l1_n, l2_n)
for i in 1:l1_n, j in 1:l2_n
    if synapse[i, j] < 0
        synapse[i, j] = 0
    end
end
synapse

5×3 Matrix{Float64}:
 1.10621   0.0       0.0
 0.0       1.20884   0.283888
 1.19339   0.0       0.0
 1.45479   0.704365  0.168471
 0.241449  0.0       1.25201

In [22]:
st_in = rand(0:1, l1_n, Int(total_time/dt))  # array of size number_of_neurons_in_l1 * total_time_steps

5×100 Matrix{Int64}:
 0  1  0  0  0  0  1  1  0  1  1  0  1  …  0  0  0  1  1  1  1  0  0  0  0  0
 1  0  0  1  1  0  0  1  1  1  0  1  0     1  1  1  1  1  1  1  1  1  0  0  1
 0  0  0  0  1  0  1  0  1  0  1  0  1     0  1  0  0  1  1  0  1  1  0  0  0
 0  1  1  0  0  1  1  1  1  1  1  0  0     1  0  1  0  0  0  1  0  0  1  1  1
 0  0  0  1  1  1  1  0  1  1  1  0  1     1  1  1  1  0  1  0  1  0  0  0  0

In [23]:
w_in = I(l1_n)

5×5 Diagonal{Bool, Vector{Bool}}:
 1  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  ⋅  1

In [24]:
function process!(neuron, input_spikes, synapse)
    for i in 1:total_time
        if i == 0  # first step
            neuron.V = dot(synapse, input_spikes[:, i]) * alpha
            neuron.spike_out[i] = 0
        else
            neuron.V = neuron.V * alpha + dot(synapse, input_spikes[:, i])
            if neuron.V >= θ
                neuron.V = ρ
                neuron.spike_out[i] = 1
            else
                neuron.spike_out[i] = 0
            end
        end
    end
    
end

process! (generic function with 1 method)

In [25]:
out_l1 = []
for i in 1:l1_n
    process!(l1[i], st_in, w_in[:, i])
    push!(out_l1, l1[i].spike_out)
end

In [26]:
out_l1

5-element Vector{Any}:
 Float32[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 Float32[1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0  …  -9.11071f14, 4.583f-41, 4.8f-44, 0.0, 5.2f-44, 0.0, -9.110753f14, 4.583f-41, -9.110796f14, 4.583f-41]
 Float32[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0  …  4.6f-44, 0.0, -9.166287f14, 4.583f-41, -9.16633f14, 4.583f-41, 4.8f-44, 0.0, 5.2f-44, 0.0]
 Float32[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0f-45, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 Float32[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [27]:
out_l2 = []
for i in 1:l2_n
    process!(l2[i], st_in, synapse[:, i])
    push!(out_l2, l2[i].spike_out)
end

In [28]:
out_l2

3-element Vector{Any}:
 Float32[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0f-45, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 Float32[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0  …  1.0f-45, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 Float32[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]